In [1]:
'''
RNN for MNIST digits classification
97.7% test accuracy in 20epochs
75K adjustable parameters
input -> 256 -> 10

https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist



In [2]:
!python --version
!pip list | grep tensorflow

Python 3.6.9
tensorflow                    2.4.0          
tensorflow-addons             0.8.3          
tensorflow-datasets           4.0.1          
tensorflow-estimator          2.4.0          
tensorflow-gcs-config         2.4.0          
tensorflow-hub                0.10.0         
tensorflow-metadata           0.26.0         
tensorflow-privacy            0.2.2          
tensorflow-probability        0.11.0         


## Clean Data

In [3]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# resize and normalize
#image_size = x_train.shape[1]
#print(x_train.shape)

x_train = np.reshape(x_train,[x_train.shape[0], -1, 1])
x_test = np.reshape(x_test,[x_test.shape[0], -1, 1])

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (60000, 784, 1)
y_train (60000, 10)
x_test (10000, 784, 1)
y_test (10000, 10)


In [4]:
print(x_train[0].shape)
print(x_train[0][0])

(784, 1)
[0.]


## Setup Model

In [5]:
from keras import initializers

# network parameters
input_shape = x_train.shape[1:]
batch_size = 128
units = 100
dropout = 0.00

# model is RNN with 256 units, input is 28-dim vector 28 timesteps
model = Sequential()
model.add(SimpleRNN(units=units,
                    #kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    #recurrent_initializer=initializers.Identity(gain=1.0),
                    #dropout=dropout,
                    input_shape=input_shape))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()
print("units", units)
print("input_shape", input_shape)
print("num_labels", num_labels)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 100)               10200     
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________
units 100
input_shape (784, 1)
num_labels 10


## Model compile and fit (train)

In [ ]:
# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks
from keras.optimizers import RMSprop

learning_rate = 1e-4 # need turn here, otherwise, no decent happens
rmsprop = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer= rmsprop,
              metrics=['accuracy'])
# train the network
model.fit(x_train, y_train, 
          epochs= 200, 
          batch_size=batch_size,
          validation_data=(x_test, y_test))



Epoch 1/200
469/469 [==============================] - 243s 516ms/step - loss: 2.3203 - accuracy: 0.1041 - val_loss: 2.3128 - val_accuracy: 0.0983
Epoch 2/200
469/469 [==============================] - 242s 516ms/step - loss: 2.3120 - accuracy: 0.0985 - val_loss: 2.3317 - val_accuracy: 0.1161
Epoch 3/200
469/469 [==============================] - 241s 514ms/step - loss: 2.3110 - accuracy: 0.1009 - val_loss: 2.3094 - val_accuracy: 0.0974
Epoch 4/200
469/469 [==============================] - 243s 518ms/step - loss: 2.3110 - accuracy: 0.1033 - val_loss: 2.3201 - val_accuracy: 0.1010
Epoch 5/200
469/469 [==============================] - 242s 516ms/step - loss: 2.3156 - accuracy: 0.1037 - val_loss: 2.3155 - val_accuracy: 0.1009
Epoch 6/200
469/469 [==============================] - 242s 515ms/step - loss: 2.3093 - accuracy: 0.1065 - val_loss: 2.3039 - val_accuracy: 0.1010
Epoch 7/200
469/469 [==============================] - 244s 520ms/step - loss: 2.3093 - accuracy: 0.1034 - val_loss: 2

## Model pred

In [ ]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))